<a href="https://colab.research.google.com/github/aabelatilaw/COVID-19-Prediction-Using-Artificial-Intelligence/blob/main/Country_Wise_Death_Cases_Due_to_COVID_19_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [16]:
import pandas as pd
import numpy as np
import tensorflow as tf
keras = tf.keras
from sklearn.preprocessing import RobustScaler
import plotly.graph_objects as go
physical_devices = tf.config.experimental.list_physical_devices('GPU')

# Check if any GPUs were found
if physical_devices:
    # Set memory growth for the first GPU
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
else:
    # Print a warning if no GPUs were found
    print("No GPUs found. TensorFlow will run on CPU.")



No GPUs found. TensorFlow will run on CPU.


# Importing Data

In [17]:
#death_df = pd.read_csv(CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
from google.colab import files
uploaded = files.upload()


Saving time_series_covid19_deaths_global.csv to time_series_covid19_deaths_global (2).csv


In [18]:
death_df = pd.read_csv ('time_series_covid19_deaths_global.csv')
death_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7896,7896,7896,7896,7896,7896,7896,7896,7896,7896
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3598,3598,3598,3598,3598,3598,3598,3598,3598,3598
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6881,6881,6881,6881,6881,6881,6881,6881,6881,6881
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,165,165,165,165,165,165,165,165,165,165
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,1933,1933,1933,1933,1933,1933,1933,1933,1933,1933


# Data Pre-Processing

In [20]:
death_df = pd.read_csv('time_series_covid19_deaths_global.csv')
death_df = death_df.groupby("Country/Region").sum().reset_index()
Country_death = death_df['Country/Region'].values
death_df = death_df.drop(['Lat', 'Long', 'Country/Region'], axis=1)
death_df = death_df[death_df.columns].T
death_df.columns = Country_death
Date_index = death_df.index

# Convert all columns to numeric, coercing errors to NaN
for col in death_df.columns:
    death_df[col] = pd.to_numeric(death_df[col], errors='coerce')

death_df = death_df.diff().fillna(0)
death_df = death_df.abs()
death_df.head()

,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
Province/State,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1/22/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1/23/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1/24/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1/25/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Scalling the data

In [25]:
sc = RobustScaler()
death_df_scaled = sc.fit_transform(death_df)
death_df_scaled = pd.DataFrame(death_df_scaled)

# Enter the number of days you want to predict

In [40]:
DAYS_TO_PREDICT = int(input('Enter Number Of Days You Want to Predict : '))
# Replace 'closed' with 'inclusive' and use 'right' for the desired behavior
predict_death_index = pd.date_range(start=Date_index[-1], periods=DAYS_TO_PREDICT + 1, inclusive='right')
predict_death_index = predict_death_index.strftime('%Y/%m/%d')

Enter Number Of Days You Want to Predict : 1


# Creating class for building model

In [41]:
class Corona_death_model():
    def create_dataset(self, dataset, time_steps=1, column = 0):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_steps-1):
          a = dataset[i:(i+time_steps), column]
          dataX.append(a)
          dataY.append(dataset[i + time_steps, column])
        return np.array(dataX), np.array(dataY)

    def Model(self,dframe,column):
        df = dframe.values
        df = df.astype('float32')

        time_steps = 1
        trainX, trainY = self.create_dataset(df, time_steps, column)
        trainX = np.reshape(trainX, (trainX.shape[0],1,trainX.shape[1]))


        keras.backend.clear_session()
        tf.random.set_seed(42)
        np.random.seed(42)
        death_model = keras.models.Sequential([
            keras.layers.InputLayer(input_shape=[None, 1]),
            keras.layers.Conv1D(filters=16, kernel_size=2,
                      strides=1, padding="causal",
                      activation="relu",
                      ),
            keras.layers.LSTM(30,kernel_initializer = 'he_normal',
                              return_sequences = True),
            keras.layers.LSTM(30,kernel_initializer = 'he_normal',
                              return_sequences = True),
            keras.layers.Dense(15,kernel_initializer = 'he_normal',
                                             activation = 'relu'),
            keras.layers.Dense(15,kernel_initializer = 'he_normal',
                                             activation = 'relu'),
            keras.layers.Dense(1,activation = 'linear',kernel_initializer = 'he_normal'),
            keras.layers.Lambda(lambda x: x * 200)
             ])

        optimizer = keras.optimizers.Adadelta(lr = 0.001)

        death_model.compile(loss=['mae'],optimizer=optimizer)

        death_model.fit(trainX, trainY, epochs=100,
                          batch_size = 10,verbose = 0)

        death_test_seq  = trainX[-1:]
        death_test_seq = death_test_seq
        death_test_seq = death_test_seq.reshape(len(death_test_seq), time_steps, 1)
        death_preds = []

        for _ in range (DAYS_TO_PREDICT):
            death_pred = death_model.predict(death_test_seq)
            death_preds.append(death_pred)
            death_new_seq = death_pred
            death_test_seq = death_new_seq
            death_test_seq = death_test_seq.reshape(len(death_test_seq ), time_steps, 1)



        return death_preds

# Calling the class and predicting the cases for every individual country

In [42]:
class Corona_death_model():
    def Model(self,dframe,column):
        optimizer = keras.optimizers.Adadelta(learning_rate=0.001)

NN = Corona_death_model()
death_predictions = []
for i in range(len(Country_death)):
    result = NN.Model(death_df_scaled,i)
    death_predictions.append(result)





        # ... (rest of the code) ...

In [43]:
death_predictions = np.array(death_predictions) # shape (201, 30, 1) or (201, 30) depending on your model output
#death_predictions = death_predictions.reshape((DAYS_TO_PREDICT, len(Country_death))) # shape (30, 201)


death_predictions = (np.array(death_predictions).transpose())
death_predictions = death_predictions.reshape((DAYS_TO_PREDICT, len(Country_death))) # shape (30, 201)
#death_predictions = death_predictions.reshape(((DAYS_TO_PREDICT),len(Country_death)))
death_predictions = sc.inverse_transform(death_predictions)
death_predictions = pd.DataFrame(death_predictions)
death_predictions = death_predictions.abs()
death_predictions['Dates'] = predict_death_index
death_predictions.set_index('Dates', inplace = True)
death_predictions.columns = Country_death
death_predictions.head()

,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
Dates,,,,,,,,,,,,,,,,,,,,,
2023/03/10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Visualizing The Death Cases For A Country As Per The User's Choice

### List of Infected Countries

In [45]:
Infected_Countries = pd.DataFrame(Country_death)
Infected_Countries.columns = ['Countries']
pd.set_option("display.max_rows", None)
Infected_Countries

,Countries
0,Afghanistan
1,Albania
2,Algeria
3,Andorra
4,Angola
5,Antarctica
6,Antigua and Barbuda
7,Argentina
8,Armenia
9,Australia


### Select The Country

In [46]:
Country = input('Enter the name of the country from above list you would like to see the daily cases till now : ')

Enter the name of the country from above list you would like to see the daily cases till now : Ethiopia


## Till Date Daily Cases of Death due to coronavirus

In [47]:
fig = go.Figure()
fig.update_layout(template='plotly_dark',title={'text': 'Death Daily Cases','y':0.9,'x':0.5,'xanchor': 'center',
                                                'yanchor': 'top'},
                                    xaxis_title="Dates",
                                    yaxis_title="Number Of Cases")
fig.add_trace(go.Scatter(x=death_df.index,
                         y=death_df[Country],
                         mode='lines+markers',
                         line=dict(color='red', width=2)))

## Predictions of death cases due to coronavirus

In [48]:
fig = go.Figure()
fig.update_layout(template='plotly_dark',title={'text': 'Death Cases Predictions','y':0.9,'x':0.5,'xanchor': 'center',
                                                'yanchor': 'top'},xaxis_title="Dates",yaxis_title="Number Of Cases")
fig.update_xaxes(tickangle=90, showticklabels = True, type = 'category')
fig.add_trace(go.Scatter(x=death_predictions.index,
                         y=death_predictions[Country],
                         mode='lines+markers',
                         line=dict(color='turquoise', width=2)))